## 0.Going Modular
In this lab we are going to convert custom data make them modular

### 0.1Importing data

In [44]:
import os
import requests
import zipfile
from pathlib import Path

In [45]:
p = Path('.')
[x for x in p.iterdir() if x.is_dir()]

[PosixPath('.config'),
 PosixPath('data'),
 PosixPath('going_modular'),
 PosixPath('sample_data')]

In [46]:
# Setup path to data folder
data_path = Path('data/')
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
  print(f"{image_path} already exist. Exit downloading")
else:
  print(f"Creating image folder at {image_path}")
  image_path.mkdir(parents=True, exist_ok=True)

# Download pizza, steak, sushi data
with open(data_path / "pizza_steak_sushi.zip","wb") as f:
  request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  print('Downloading pizza steak and sushi')
  f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
  print("unzipping file")
  zip_ref.extractall(image_path)

data/pizza_steak_sushi already exist. Exit downloading
unzipping file


In [47]:
# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")

In [48]:
#set up training and testing path
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

In [49]:
#transformation
from torchvision import transforms

data_transform = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.ToTensor()
])
data_transform

Compose(
    Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
)

## 1.Create datasets and dataloaders
* we are using magic command `%%writefile` this convert code to script
* we are writing docstring in Google python docstring style

In [50]:
os.makedirs('going_modular', exist_ok=True)

In [51]:
%%writefile going_modular/data_setup.py
"""
contain functionality for creating pytorch DataLoaders for
image classification data.
"""
import os

from torchvision import datasets,transforms
from torch.utils.data import DataLoader

NUM_WORKER = os.cpu_count()

def create_dataloaders(train_dir,
                      test_dir,
                      transform,
                      batch_size: int,
                      num_worker: int=NUM_WORKER):
  '''Creating training and testing dataloaders

  take training directory and testing directory path and turn them into
  pytorch dataset and finally pytorch dataloaders

  Args:
    train_dir: path to training directory
    test_dir: path to testing directory
    transform: torchvision transform-> to apply in training and testing data
    batch_size: no of sample in per batch in each dataloader
    num_worker: no of workers per dataloader

  Returns:
    A tuple of (train_dataloader, test_dataloader, class_names)
    where `class_names` is list of target label

    Example Usage:
      train_dataloader, test_dataloader, clss_names = \
        = create_dataloaders(train_dir= path/to/train_dir,
                            test_dir= path/to/test_dir,
                            transform= some_transformation,
                            batch_size= 32,
                            num_worker= 4)

  '''

  #take and transform data aka `ImageFolder`
  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  # get class name
  class_names = train_data.classes

  # turn image into dataloaders aka `python iterable`
  train_dataloader = DataLoader(
      dataset=train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_worker,
      pin_memory=True)

  test_dataloader = DataLoader(
      dataset=test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_worker,
      pin_memory=True
  )

  return train_dataloader, test_dataloader, class_names

Overwriting going_modular/data_setup.py


In [52]:
from going_modular import data_setup

train_dataloaders, test_dataloaders, class_name = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir= test_dir,
    transform= data_transform,
    batch_size= 32
)

train_dataloaders, test_dataloaders, class_name

(<torch.utils.data.dataloader.DataLoader at 0x7dd294924580>,
 ['pizza', 'steak', 'sushi'])

## 2.Making model builder

In [53]:
%%writefile going_modular/model_builder.py
'''
Contain pytorch model code to instantiate a TinyVGG model
'''

import torch
from torch import nn

class TinyVGG(nn.Module):
  """Creates the TinyVGG architecture.

  Replicate a TinyVGG architecture from CNN Explainer with little modification

  Args:
    input_shape: integer indicating no of input channels.
    output_shape: integer indicating no of output channels.
    hidden_unit: integer indicating no of hidden units.

  """
  def __init__(self,
               input_shape: int,
               output_shape: int,
               hidden_unit: int):
    super().__init__()

    self.conv_layer_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_unit,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_unit,
                  out_channels=hidden_unit,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.conv_layer_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_unit,
                  out_channels=hidden_unit,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_unit,
                  out_channels=hidden_unit,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_unit * 16 * 16, out_features=output_shape)
    )

  def forward(self, x):
    return self.classifier(self.conv_layer_2(self.conv_layer_1(x)))

Overwriting going_modular/model_builder.py


In [54]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [55]:
from going_modular import model_builder

model_0 = model_builder.TinyVGG(input_shape=3, output_shape=len(class_name), hidden_unit=10).to(device)
model_0

TinyVGG(
  (conv_layer_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_layer_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2560, out_features=3, bias=True)
  )
)

In [56]:
try:
  import torchinfo
except:
  !pip install torchinfo
  import torchinfo

In [57]:
from torchinfo import summary

summary(model=model_0, input_size=(32, 3, 64, 64))

Layer (type:depth-idx)                   Output Shape              Param #
TinyVGG                                  [32, 3]                   --
├─Sequential: 1-1                        [32, 10, 32, 32]          --
│    └─Conv2d: 2-1                       [32, 10, 64, 64]          280
│    └─ReLU: 2-2                         [32, 10, 64, 64]          --
│    └─Conv2d: 2-3                       [32, 10, 64, 64]          910
│    └─ReLU: 2-4                         [32, 10, 64, 64]          --
│    └─MaxPool2d: 2-5                    [32, 10, 32, 32]          --
├─Sequential: 1-2                        [32, 10, 16, 16]          --
│    └─Conv2d: 2-6                       [32, 10, 32, 32]          910
│    └─ReLU: 2-7                         [32, 10, 32, 32]          --
│    └─Conv2d: 2-8                       [32, 10, 32, 32]          910
│    └─ReLU: 2-9                         [32, 10, 32, 32]          --
│    └─MaxPool2d: 2-10                   [32, 10, 16, 16]          --
├─Sequentia

## 3.Creating  test fn
* train_step() , test_step() fn and combine

In [58]:
%%writefile going_modular/engine.py
'''
Contain fn for training and testing a pytorch module
'''

import torch

from tqdm.auto import tqdm
from typing import List, Dict, Tuple

def train_step(model,
               dataloaders,
               loss_fn,
               optimizer,
               device) -> Tuple[float, float]:
  """Train a pytorch model for single epochs

  Turn the model to training mode and run through require training
  forward pass, loss calculation and optimizer step.

  Args:
    model: model to be train
    dataloaders: train dataloaders
    loss_fn: calculate loss
    optimizer: optimize the loss
    device: which device to train

  Returns:
    return tuple of train_loss, train_acc

  """

  train_loss, train_acc = 0, 0

  model.train()

  for X, y in dataloaders:
    X, y = X.to(device), y.to(device)
    #forward
    y_preds = model(X)
    #loss
    loss = loss_fn(y_preds, y)
    train_loss += loss.item()
    #optimizer.zero_grad, backpropagation, optimizer.step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #accuracy
    y_pred_class = torch.argmax(torch.softmax(y_preds, dim=1), dim=1)
    train_acc += (y_pred_class == y).sum().item()/len(y_preds)

  train_loss /= len(dataloaders)
  train_acc /= len(dataloaders)
  return train_loss, train_acc


def test_step(model,
              dataloaders,
              loss_fn,
              device) -> Tuple[float, float]:
  """Test a model for single epochs

  Turn model to eval mode and perform forward pass and calc loss

  Args:
    model= model to test
    dataloaders= test_dataloders
    loss_fn = to calc loss
    device= device to perform calc

  Returns:
    returns tuples of test_loss and test_acc

  """
  test_loss, test_acc = 0, 0

  model.eval()
  with torch.inference_mode():
    for X, y in dataloaders:
      X, y = X.to(device), y.to(device)
      #forward
      y_preds = model(X)
      #loss
      loss = loss_fn(y_preds, y)
      test_loss += loss.item()

      acc = torch.argmax(torch.softmax(y_preds, dim=1), dim=1)
      test_acc += (acc==y).sum().item()/len(y_preds)

  test_loss /= len(dataloaders)
  test_acc/= len(dataloaders)

  return test_loss, test_acc


def train(model,
          train_dataloaders,
          test_dataloaders,
          loss_fn,
          optimizer,
          epochs,
          device) -> Dict[str, List]:
  """Combine both train_step() and test_step()

  Train and Test a model for no of epochs

  Args:
    model: model to train
    train_dataloaders: train_dataloader
    test_dataloaders: test_dataloader
    loss_fn: calculate the loss
    optimizer: minimize the loss
    epochs: no of times model should train
    device: device where model should train

  Return:
    return a dictionary of train_loss,train_acc,test_loss,test_acc
  """

  results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }

  for epoch in tqdm(range(epochs)):

    train_loss, train_acc = train_step(
        model=model,
        dataloaders= train_dataloaders,
        loss_fn= loss_fn,
        optimizer=optimizer,
        device=device
    )

    test_loss, test_acc = test_step(
        model= model,
        dataloaders= test_dataloaders,
        loss_fn = loss_fn,
        device= device
    )

    # Print out what's happening
    print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
    )

      # Update results dictionary
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

  # Return the filled results at the end of the epochs
  return results

Overwriting going_modular/engine.py


In [59]:

from going_modular.engine import train

optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.001)

train(model= model_0,
      train_dataloaders=train_dataloaders,
      test_dataloaders=test_dataloaders,
      loss_fn=nn.CrossEntropyLoss(),
      optimizer=optimizer,
      epochs= 5,
      device=device)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.1038 | train_acc: 0.3047 | test_loss: 1.0987 | test_acc: 0.2604
Epoch: 2 | train_loss: 1.0943 | train_acc: 0.4258 | test_loss: 1.0999 | test_acc: 0.2604
Epoch: 3 | train_loss: 1.0930 | train_acc: 0.4258 | test_loss: 1.1012 | test_acc: 0.2604
Epoch: 4 | train_loss: 1.0929 | train_acc: 0.4258 | test_loss: 1.1024 | test_acc: 0.2604
Epoch: 5 | train_loss: 1.1040 | train_acc: 0.3047 | test_loss: 1.1039 | test_acc: 0.2604


{'train_loss': [1.1037781536579132,
  1.094342589378357,
  1.092969685792923,
  1.0928970128297806,
  1.1040384024381638],
 'train_acc': [0.3046875, 0.42578125, 0.42578125, 0.42578125, 0.3046875],
 'test_loss': [1.0987149477005005,
  1.0999411344528198,
  1.101176659266154,
  1.1024039189020793,
  1.10391100247701],
 'test_acc': [0.2604166666666667,
  0.2604166666666667,
  0.2604166666666667,
  0.2604166666666667,
  0.2604166666666667]}

## 4.Saving model

In [60]:
%%writefile going_modular/utils.py


import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)

  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

Writing going_modular/utils.py


## 5.Train,eval and save model

In [61]:
%%writefile going_modular/train.py
import os
import torch
from torch import nn

from torchvision import transforms
from timeit import default_timer as timer

import data_setup, engine, model_builder, utils

# hyperparameters
EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Setup directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
                                     transforms.Resize((64, 64)),
                                     transforms.ToTensor()
])

# Create DataLoader's and get class_names
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=data_transform,
                                                                               batch_size=BATCH_SIZE)


#model
model = model_builder.TinyVGG(input_shape=3,
                              output_shape=len(class_names),
                              hidden_unit=10).to(device)
#loss_fn and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(),
                             lr=LEARNING_RATE)

start_time = timer()
engine.train(model= model,
      train_dataloaders=train_dataloader,
      test_dataloaders=test_dataloader,
      loss_fn=loss_fn,
      optimizer=optimizer,
      epochs= EPOCHS,
      device=device)

end_time = timer()
print(f"total time: {end_time - start_time:.3f}")

# Save the model to file
utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_going_modular_script_mode_tinyvgg_model.pth")

Writing going_modular/train.py


In [62]:
!python going_modular/train.py

  0% 0/5 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.1088 | train_acc: 0.3008 | test_loss: 1.1051 | test_acc: 0.1979
 20% 1/5 [00:01<00:05,  1.28s/it]Epoch: 2 | train_loss: 1.1050 | train_acc: 0.2930 | test_loss: 1.1458 | test_acc: 0.1979
 40% 2/5 [00:02<00:03,  1.03s/it]Epoch: 3 | train_loss: 1.0947 | train_acc: 0.3242 | test_loss: 1.1465 | test_acc: 0.2604
 60% 3/5 [00:02<00:01,  1.06it/s]Epoch: 4 | train_loss: 1.1221 | train_acc: 0.4062 | test_loss: 1.1538 | test_acc: 0.2292
 80% 4/5 [00:03<00:00,  1.10it/s]Epoch: 5 | train_loss: 1.0836 | train_acc: 0.5000 | test_loss: 1.1009 | test_acc: 0.2396
100% 5/5 [00:04<00:00,  1.08it/s]
total time: 4.649
[INFO] Saving model to: models/05_going_modular_script_mode_tinyvgg_model.pth
